In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
data = pd.read_csv("A:\Download\manhattan.csv")

In [3]:
data.head()

,GRID_ID,idle_count,relief_stands,geometry,dropoff_co,parks,rest_cheap,violations
0,AS-54,0.0,0,POLYGON ((-74.04589966952779 40.69033272469935...,0.0,0.0,0,0.0
1,AS-55,0.0,0,POLYGON ((-74.04589966952779 40.69033272469935...,0.0,0.0,0,0.0
2,AT-53,0.0,0,POLYGON ((-74.04109861205649 40.70049294329179...,0.5,0.0,0,0.0
3,AT-54,0.0,0,POLYGON ((-74.04146718007551 40.69659112724123...,0.0,0.0,0,0.0
4,AT-55,0.0,0,POLYGON ((-74.04388559622677 40.69018482394773...,0.0,0.0,0,0.0


In [4]:
subdat = data[["GRID_ID", "idle_count", "dropoff_co", "parks", "rest_cheap", "violations"]]

In [36]:
dat = subdat[["idle_count", "dropoff_co", "parks", "rest_cheap", "violations"]]

In [37]:
dat = (dat - dat.min()) / (dat.max() - dat.min())

In [42]:
dat["GRID_ID"] = subdat["GRID_ID"]

In [93]:
scores_m = np.zeros([dat.shape[0],50])
sort_list = ["violations", "idle_count", "dropoff_co", "rest_cheap", "parks"]
np.random.seed(100)
for _ in range(50):
    weights = np.array(sorted(np.random.dirichlet(np.ones(5)))[::-1])
    vals = dat[sort_list].values
    scores = vals.dot(weights)
    scores_m[:,_] = vals.dot(weights).reshape(vals.shape[0])
    print "score " + str(_) + " is:"
    for i,v in enumerate(sort_list):
        print v + " is: " + str(weights[i])
    print "end \n"

score 0 is:
violations is: 0.527675618273
idle_count is: 0.222061638472
dropoff_co is: 0.156512657033
rest_cheap is: 0.0924102738331
parks is: 0.00133981238829
end 

score 1 is:
violations is: 0.437916683527
idle_count is: 0.278339616274
dropoff_co is: 0.214438173667
rest_cheap is: 0.0368303281448
parks is: 0.0324751983858
end 

score 2 is:
violations is: 0.734443291996
idle_count is: 0.0820937320416
dropoff_co is: 0.0776723327425
rest_cheap is: 0.0678296663018
parks is: 0.0379609769185
end 

score 3 is:
violations is: 0.498239701288
idle_count is: 0.219488758936
dropoff_co is: 0.216325766619
rest_cheap is: 0.0415006027559
parks is: 0.0244451704018
end 

score 4 is:
violations is: 0.495113750549
idle_count is: 0.299439957107
dropoff_co is: 0.0994330051621
rest_cheap is: 0.0720774353215
parks is: 0.0339358518606
end 

score 5 is:
violations is: 0.671019186478
idle_count is: 0.197141934401
dropoff_co is: 0.122932447763
rest_cheap is: 0.00649582781212
parks is: 0.00241060354552
end 

scor

In [96]:
for _ in range(10):
    data["score_"+ str(_)] = scores_m[:,_]

In [99]:
data.to_csv("withscores.csv")

### Linear model with logistic regression

In [88]:
import geopandas as gpd
from fiona.crs import from_epsg

In [89]:
dat = gpd.GeoDataFrame.from_csv("manhattan.csv")

In [131]:
dat["ave_score"] = dat[["score_" + str(i) for i in range(10)]].mean(axis=1)

In [132]:
dat[["parks", "dropoff_co","GRID_ID"]][dat["parks"] != 0].sort_values(["dropoff_co"], ascending=False).head(20).to_csv("parks.csv")

In [133]:
dat.sort_values("ave_score", ascending=False).to_csv("withscores.csv")

In [90]:
request_dat = gpd.GeoDataFrame.from_file("stand_requests\stand_requests.shp")

In [91]:
request_dat = request_dat.set_geometry("geometry")
request_dat.crs = from_epsg(4326)

In [92]:
haxgon =  gpd.GeoDataFrame.from_file("Hexagon_2km2/Hexagon_clipped/hexagons_clipped.shp")

In [93]:
haxgon = haxgon.set_geometry("geometry")
haxgon.crs = from_epsg(2263)
haxgon = haxgon.to_crs(epsg=4326)

In [94]:
request_dat["GRID_ID"] = request_dat.apply(lambda x: haxgon[haxgon.geometry.contains(x.geometry)]["GRID_ID"].values[0], axis=1)

In [95]:
manhattan = request_dat[request_dat["borough"] == "Manhattan"]

In [96]:
dat_sort = dat.sort_values("idle_count", ascending=False)

In [98]:
features = ["parks", "rest_cheap"]

In [99]:
request_ = dat_sort.merge(request_dat, on="GRID_ID")[features + ["GRID_ID"]]

In [100]:
dat_ = dat_sort[features + ["GRID_ID", "relief_stands"]]

In [101]:
request_["flag"] = 1

In [102]:
flags1 = dat_[dat["relief_stands"] != 0][features + ["GRID_ID"]]

B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [103]:
flags1["flag"] = np.append(np.ones(10), np.zeros(flags1.shape[0]-10))

In [104]:
final = flags1.append(request_).drop_duplicates()

In [119]:
y_train = final["flag"].values
x_train = final[["rest_cheap", "parks"]].values

In [106]:
from sklearn import linear_model

In [120]:
for c in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]:
    lgc = linear_model.LogisticRegression(C=c)
    lgc.fit(x_train, y_train)
    print ("c is: " + str(c) + " score is: " + str(lgc.score(x_train, y_train)))

c is: 1e-05 score is: 0.536585365854
c is: 0.0001 score is: 0.536585365854
c is: 0.001 score is: 0.536585365854
c is: 0.01 score is: 0.536585365854
c is: 0.1 score is: 0.536585365854
c is: 1 score is: 0.609756097561
c is: 10.0 score is: 0.634146341463
c is: 100.0 score is: 0.634146341463
c is: 1000.0 score is: 0.634146341463
c is: 10000.0 score is: 0.634146341463
c is: 100000.0 score is: 0.634146341463
c is: 1000000.0 score is: 0.634146341463


In [130]:
y_train

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.])

In [121]:
lgc = linear_model.LogisticRegression(C=10)
lgc.fit(x_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [124]:
prob = lgc.predict_proba(dat_[features])[:,1]

In [151]:
f_dat = dat_
f_dat["prob"] = prob
f_dat["violations"] = dat["violations"]
f_dat["idle_count"] = dat["idle_count"]

B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [126]:
import pandas as pd
import numpy as np

In [127]:
f_dat.reset_index(drop=True, inplace=True)

In [132]:
neighbour = np.load("neighbour.npy")

In [133]:
name = pd.read_csv("manhattan_grids_final.csv")

In [152]:
f_dat_final = f_dat.merge(name, on="GRID_ID")

In [7]:
import pulp

In [153]:
trs_model = pulp.LpProblem("trs Problem", pulp.LpMaximize)

In [154]:
x = pulp.LpVariable.dict('x_%s', range(f_dat_final.shape[0]))
y = pulp.LpVariable.dict('y_%s', range(f_dat_final.shape[0]), lowBound=0, upBound=1, cat="Binary")

In [155]:
trs_model += sum([f_dat_final.loc[i, "violations"] * x[i] * f_dat_final.loc[i, "prob"] for i in range(f_dat_final.shape[0])])

In [156]:
trs_model += sum([y[i] for i in range(f_dat_final.shape[0])]) <= 20
for i in range(f_dat_final.shape[0]):
    zero = np.zeros(f_dat_final.shape[0])
    zero[i] = 1
    trs_model += sum(x[i]*zero[i] for i in range(f_dat_final.shape[0])) <= f_dat_final.loc[i, "idle_count"]
    trs_model += sum(x[i]*zero[i] for i in range(f_dat_final.shape[0])) <= 0.8*y[i] + 0.2*sum([y[f_dat_final[f_dat_final["num"]==j].index.values[0]] for j in np.where(neighbour[f_dat_final.loc[i, "num"], :] == 1)[0]])

In [157]:
trs_model.solve()

1

In [158]:
ids = []
for v in trs_model.variables():
    if v.name.startswith("y") & (v.varValue == 1):
        ids.append(f_dat.loc[int(v.name[2:]), "GRID_ID"])

In [159]:
ids

['BQ-17',
 'BL-33',
 'BI-42',
 'BQ-22',
 'BK-47',
 'BJ-34',
 'BR-28',
 'BK-42',
 'BI-46',
 'BI-27',
 'BG-50',
 'BN-38',
 'BT-11',
 'BJ-33',
 'BJ-48',
 'BP-34',
 'BR-31',
 'BM-32',
 'BO-24',
 'BO-22']